In [1]:
# Utils
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt

import numpy as np
import time
import pickle
import re, sys, unidecode

from sklearn.metrics import accuracy_score
from IPython.display import display, Markdown, Latex

import shutil
import os
import scipy

import scikitplot as skplt

from tensorflow.contrib.learn import DNNClassifier



/root/anaconda3/envs/tensorflow/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Parameters

In [2]:
root_path = "../"
path_model = root_path + 'models/seccionV2'
#features_path = root_path + 'data/features/data_tfid_hash28_n1000_SVD2.p'
#features_path = root_path + 'data/features/dataV2_tfid_hash28_n10000_svd1000_sin_stemmig.p'

#features_path = root_path + 'data/features/dataV2_tfid_hash28_n10000_svd1000_sin_stemmig.p'
features_path = root_path + 'data/features/dataV2_tfid_hash28_n10000_svd1000_bigram_sin_stemmig.p'
#features_path = root_path + 'data/features/dataV2_tfid_hash28_n10000_svd1000_bigram_con_stemmig.p'

delete_old_model = False
if delete_old_model:
    try:
        os.system("rm -rf "+path_model)
        os.system("mkdir "+path_model)
    except:
        print("error")

# Load Data

In [3]:
#features_path = root_path + 'data/features/dataV2_tfid_hash28_n10000_svd1000_sin_stemmig.p'

X, y1, y2, y3 = pickle.load( open( features_path, "rb" ) )


In [4]:
y1.values

array(['mundo', 'pais', 'mundo', ..., 'pais', 'pais', 'deportes'],
      dtype=object)

# pre procesing

Cleaning data, select clases

In [5]:
y2 = np.array(y2)
y_2 = list(map(lambda x: unidecode.unidecode(x) if x!=None else None, y2))
y_2 = np.array(y_2)


y1 = y1.values

# Feature Extraccion

### Data representation

TFID calculation

### filter clases

In [6]:
labels = list(set(y1))
labels

['tecnologia',
 'deportes',
 'economia',
 'pais',
 'sociedad',
 'entretencion',
 'cultura',
 'mundo']

In [7]:
def filterClases(X,y,umbral):
    labels = list(set(y))
    sizes = [ [label,y[y== label].shape[0]] for label in labels ]
    filter_sizes = list(filter(lambda x:x[1]>umbral ,sizes ))
    names_clases = set( map(lambda x:x[0], filter_sizes ) )
    index = list(map(lambda x: {x}.issubset(names_clases), y ))
    y = y[index]
    X = X[np.nonzero(index)]
    return X,y



In [8]:
y_temas = {}
X_temas = {}
labels_temas = {}
labels_temas_before = {}
umbral_ejemplos = 100

for key in labels:
    index = y1 == key
    y_22 = y_2[index]
    X2 = X[np.nonzero(index)]

    index = y_22 != None
    y_22 = y_22[index]
    X2 = X2[np.nonzero(index)]
    X_temas[key],y_temas[key] = filterClases(X2,y_22,umbral_ejemplos)
    
    
    labels_temas_before = list(set(y_22))
    labels_temas[key] = list(set(y_temas[key]))
    
    print("\n"+key + " ,total : " + str(len(labels_temas[key])) \
          +" ,deleted : "+ str( len(labels_temas_before) - len(labels_temas[key] )) )
    print(labels_temas[key])



tecnologia ,total : 4 ,deleted : 3
['internet', 'redes sociales', 'inventos', 'industria']

deportes ,total : 39 ,deleted : 8
['eurocopa', 'copa davis', 'automovilismo', 'polideportivo', 'natacion', 'artes marciales', 'velerismo', 'golf', 'rally', 'mundiales de futbol', 'balonmano', 'juegos olimpicos', 'atletismo', 'al aire libre', 'juegos olimpicos de invierno', 'tenis de mesa', 'copa libertadores', 'torneo de clausura', 'hockey cesped', 'boxeo', 'londres ', 'futbol', 'torneo de apertura', 'triatlon', 'brasil ', 'gobierno', 'hockey patin', 'baloncesto', 'rusia ', 'gimnasia', 'ciclismo', 'fuera de juego', 'motociclismo', 'rugby', 'olimpismo', 'voleibol', 'copa america', 'paralimpicos', 'tenis']

economia ,total : 16 ,deleted : 2
['impuestos', 'crecimiento', 'bolsas', 'crisis financiera', 'retail', 'materias primas', 'presupuesto', 'banco central', 'organismos internacionales', 'divisas', 'servicios financieros', 'empresas', 'sueldo minimo', 'competitividad', 'sectores productivos', 's

### Output

In [9]:
np.random.seed(42)
train_fraction = 0.8

X_train = {}
y_train = {}
X_test = {}
y_test = {}
train_indices = {}
test_indices = {}

for key in labels:
    train_indices[key] = np.random.choice(X_temas[key].shape[0], round(train_fraction*X_temas[key].shape[0]), replace=False)
    test_indices[key] = np.array(list(set(range(X_temas[key].shape[0])) - set(train_indices)))

    X_train[key] = X_temas[key][train_indices[key]]
    y_train[key] = y_temas[key][train_indices[key]]
    X_test[key] = X_temas[key][test_indices[key]]
    y_test[key] = y_temas[key][test_indices[key]]

In [10]:
weights_train = {}
weights_test = {}

np.random.seed(42)

for key in labels:
    labelsTmp = list(set(y_train[key]))
    nClasses = len(labelsTmp)

    sizes = {label: y_train[key][y_train[key] == label].shape[0] for label in labelsTmp}
    weights = np.asarray([len(y_train[key])/(sizes[label]*nClasses) for label in y_temas[key]])

    weights = weights[:,np.newaxis]
    weights_train[key] = weights[train_indices[key]]
    weights_test[key] = weights[test_indices[key]]

    

# Model

### DNN graph generation

In [11]:
epochs = 14000

if isinstance(X_train, scipy.sparse.csr.csr_matrix):
    X_train = X_train.todense()
    X_test = X_test.todense()

# Define the test inputs
def get_train_inputs(key):    
    dataset = tf.estimator.inputs.numpy_input_fn({'x': X_train[key],'class_weights': weights_train[key]},
                                                  y_train[key][:,np.newaxis],
                                                  shuffle=True,
                                                  batch_size=64,
                                                  num_epochs=epochs)
    return dataset

def get_test_inputs(key):   
    dataset = tf.estimator.inputs.numpy_input_fn({'x': X_test[key],'class_weights': weights_test[key]},
                                                  y_test[key][:,np.newaxis],
                                                  shuffle=False)
    return dataset

### functions graph tensorflow

In [12]:


tf.set_random_seed(42)
feature_columns = [tf.contrib.layers.real_valued_column('x', dimension=1000)]

classifier = {}
path_all_models  = {}
for key in labels:
    path_all_models[key] = path_model+"/"+str(key)
    classifier[key] = DNNClassifier(                                
                               n_classes=len(labels_temas[key]), label_keys=labels_temas[key], feature_columns=feature_columns,
                               hidden_units=[2000],
                               dropout=0.5,
                               weight_column_name='class_weights',
                               model_dir = path_model+"/"+str(key),
                               config = tf.contrib.learn.RunConfig(save_checkpoints_steps = 500,
                               save_checkpoints_secs = None)                           
                              )

Instructions for updating:
When switching to tf.estimator.Estimator, use tf.estimator.RunConfig instead.
Instructions for updating:
Please switch to tf.contrib.estimator.*_head.
Instructions for updating:
Please replace uses of any Estimator from tf.contrib.learn with an Estimator from tf.estimator.*
INFO:tensorflow:Using config: {'_keep_checkpoint_every_n_hours': 10000, '_train_distribute': None, '_master': '', '_environment': 'local', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_device_fn': None, '_task_type': None, '_session_config': None, '_num_worker_replicas': 0, '_save_summary_steps': 100, '_log_step_count_steps': 100, '_task_id': 0, '_save_checkpoints_steps': 500, '_num_ps_replicas': 0, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f26f14b5710>, '_evaluation_master': '', '_save_checkpoints_secs': None, '_tf_random_seed': None, '_model_dir': '../models/seccionV2/tecnologia', '_keep_checkpoint_max':

# Training



In [13]:
#validation_monitor = {}

print("start")
start = time.time()

for key in labels:
    
    validation_monitor = tf.contrib.learn.monitors.ValidationMonitor(
        #input_fn= get_train_inputs(),
        input_fn= get_test_inputs(key),    
        every_n_steps=500,
        early_stopping_metric="accuracy",#loss
        #early_stopping_metric="loss",
        early_stopping_metric_minimize=True,
        early_stopping_rounds=2000)

    

    classifier[key].fit(input_fn=get_train_inputs(key), monitors=[validation_monitor], steps=epochs, max_steps=None)
    print("###################### "+key+" ######################")
    
end = time.time()
#print(key)
print("Training time :" + str(end - start) )


start
Instructions for updating:
Monitors are deprecated. Please use tf.train.SessionRunHook.
Instructions for updating:
When switching to tf.estimator.Estimator, use tf.estimator.EstimatorSpec. You can use the `estimator_spec` method to create an equivalent one.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ../models/seccionV2/tecnologia/model.ckpt-3001
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 3001 into ../models/seccionV2/tecnologia/model.ckpt.
INFO:tensorflow:Starting evaluation at 2019-01-10-18:28:02
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ../models/seccionV2/tecnologia/model.ckpt-3001
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-01-10-18:28:02
INFO:tensorflow:Saving dict for global step 3001: accuracy = 0.2

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ../models/seccionV2/deportes/model.ckpt-5001
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-01-10-18:28:17
INFO:tensorflow:Saving dict for global step 5001: accuracy = 0.5882506, global_step = 5001, loss = 1.79499
INFO:tensorflow:Validation (step 5002): loss = 1.79499, global_step = 5001, accuracy = 0.5882506
INFO:tensorflow:global_step/sec: 39.1392
INFO:tensorflow:loss = 1.445384, step = 5101 (2.555 sec)
INFO:tensorflow:global_step/sec: 468.044
INFO:tensorflow:loss = 1.48715, step = 5201 (0.214 sec)
INFO:tensorflow:global_step/sec: 470.037
INFO:tensorflow:loss = 2.4005141, step = 5301 (0.213 sec)
INFO:tensorflow:global_step/sec: 469.038
INFO:tensorflow:loss = 1.5382153, step = 5401 (0.213 sec)
INFO:tensorflow:Saving checkpoints for 5501 into ../models/seccionV2/deportes/model.ckpt.
INFO:tensorflow:global_step/sec: 400.755
INFO:t

INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-01-10-18:28:33
INFO:tensorflow:Saving dict for global step 3501: accuracy = 0.016420398, global_step = 3501, loss = 4.35555
INFO:tensorflow:Validation (step 3502): loss = 4.35555, global_step = 3501, accuracy = 0.016420398
INFO:tensorflow:loss = 4.2213845, step = 3501
INFO:tensorflow:global_step/sec: 41.8155
INFO:tensorflow:loss = 4.9603195, step = 3601 (0.267 sec)
INFO:tensorflow:global_step/sec: 461.206
INFO:tensorflow:loss = 3.631134, step = 3701 (0.217 sec)
INFO:tensorflow:global_step/sec: 461.396
INFO:tensorflow:loss = 4.2317753, step = 3801 (0.217 sec)
INFO:tensorflow:global_step/sec: 462.878
INFO:tensorflow:loss = 4.881641, step = 3901 (0.216 sec)
INFO:tensorflow:Saving checkpoints for 4001 into ../models/seccionV2/pais/model.ckpt.
INFO:tensorflow:global_step/sec: 319.805
INFO:tensorflow:loss = 3.8362207, step = 4001 (0.313 sec)
INFO:tensorflow:global_st

INFO:tensorflow:loss = 0.89442736, step = 4901 (0.208 sec)
INFO:tensorflow:Saving checkpoints for 5001 into ../models/seccionV2/sociedad/model.ckpt.
INFO:tensorflow:global_step/sec: 393.421
INFO:tensorflow:loss = 1.1096568, step = 5001 (0.254 sec)
INFO:tensorflow:global_step/sec: 473.341
INFO:tensorflow:loss = 1.1646317, step = 5101 (0.211 sec)
INFO:tensorflow:global_step/sec: 479.183
INFO:tensorflow:loss = 1.1172702, step = 5201 (0.209 sec)
INFO:tensorflow:global_step/sec: 468.571
INFO:tensorflow:loss = 1.2460117, step = 5301 (0.213 sec)
INFO:tensorflow:global_step/sec: 482.014
INFO:tensorflow:loss = 0.6942924, step = 5401 (0.207 sec)
INFO:tensorflow:Saving checkpoints for 5501 into ../models/seccionV2/sociedad/model.ckpt.
INFO:tensorflow:global_step/sec: 401.658
INFO:tensorflow:loss = 1.1045573, step = 5501 (0.249 sec)
INFO:tensorflow:Starting evaluation at 2019-01-10-18:28:51
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ../models/seccionV2/sociedad/

INFO:tensorflow:Saving checkpoints for 4501 into ../models/seccionV2/cultura/model.ckpt.
INFO:tensorflow:global_step/sec: 410.798
INFO:tensorflow:loss = 0.15839593, step = 4501 (0.243 sec)
INFO:tensorflow:global_step/sec: 478.547
INFO:tensorflow:loss = 0.24705857, step = 4601 (0.209 sec)
INFO:tensorflow:global_step/sec: 472.74
INFO:tensorflow:loss = 0.20555636, step = 4701 (0.211 sec)
INFO:tensorflow:global_step/sec: 472.365
INFO:tensorflow:loss = 0.25519505, step = 4801 (0.212 sec)
INFO:tensorflow:global_step/sec: 474.773
INFO:tensorflow:loss = 0.24985088, step = 4901 (0.211 sec)
INFO:tensorflow:Saving checkpoints for 5001 into ../models/seccionV2/cultura/model.ckpt.
INFO:tensorflow:global_step/sec: 406.87
INFO:tensorflow:loss = 0.18010801, step = 5001 (0.246 sec)
INFO:tensorflow:global_step/sec: 474.212
INFO:tensorflow:loss = 0.15602294, step = 5101 (0.211 sec)
INFO:tensorflow:global_step/sec: 474.927
INFO:tensorflow:loss = 0.13088158, step = 5201 (0.210 sec)
INFO:tensorflow:global_s

# Evaluation performance


In [ ]:
pred_test = {}
y_test_hat = {}
y_test_hat = {}
acc = {}

for key in labels:

    def input_fn_evaluate():
        dataset = {'x': tf.constant(X_test[key])}    
        return dataset

    pred_test[key] = classifier[key].predict_classes(input_fn=input_fn_evaluate)
    y_test_hat[key] = np.asarray([x.decode('UTF-8') for x in list(pred_test[key])])
    y_test_hat[key] = y_test_hat[key].astype(str)
    acc[key] = accuracy_score(y_true=y_test[key], y_pred=y_test_hat[key])
    

    display(Markdown('## ' + key))
    display(Markdown('## Accuracy in test: {} '.format(acc[key]*100)))    
    skplt.metrics.plot_confusion_matrix(y_test[key], y_test_hat[key],normalize=True,figsize=(20,20))
    plt.xticks(rotation=45)
    plt.show()    

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ../models/seccionV2/tecnologia/model.ckpt-5503
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [ ]:
for key in labels:
    display(Markdown('## ' + key))
    display(Markdown('## Accuracy in test: {} '.format(acc[key]*100)))    
    skplt.metrics.plot_confusion_matrix(y_test[key], y_test_hat[key],normalize=True,figsize=(20,20))
    plt.xticks(rotation=45)
    plt.show()
    
    
    #print('Accuracy in test: {}'.format(acc))


###  Save info model

In [ ]:
#pickle.dump( (path_all_models,labels_temas), open( path_model + "/info_model.p", "wb" ) )